# Embedding Model Selection

# Setup

In [1]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from utils import get_table_from_supabase

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# Retrieving Data from Supabase and Building Dataframes

I will connect to Supabase and retrieve the relevant records, in order to isolate the text data that I will use to embed and select the best model. For this purpose, I will use the 12 funder-recipient pairs that I have curated for my evaluation app.

In [2]:
#get evaluation pairs and extract ids
evaluation_pairs = get_table_from_supabase(url, key, "evaluation_pairs")
funder_ids = evaluation_pairs["funder_registered_num"].unique()
eval_recip_ids = evaluation_pairs["recipient_id"].unique()

In [ ]:
#get the 12 funders and their grants
funders = get_table_from_supabase(url, key, "funders")
funders = funders[funders["registered_num"].isin(funder_ids)]
funder_grants = get_table_from_supabase(url, key, "funder_grants")
eval_funder_grants = funder_grants[funder_grants["registered_num"].isin(funder_ids)]

#get grants for 12 funders
grant_ids = eval_funder_grants["grant_id"].unique()
grants = get_table_from_supabase(url, key, "grants")
grants = grants[grants["grant_id"].isin(grant_ids)]